In [69]:
import os
import csv
import pandas as pd
import numpy as np
import skfeature.function.sparse_learning_based.MCFS as MCFS
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import SelectFromModel, SelectKBest
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from keras.utils import to_categorical
from scipy import sparse

In [2]:
data = pd.read_csv("../feature_selection.csv")

In [3]:
data = data.drop("Unnamed: 0", axis = 1)

In [4]:
data.columns

Index(['SEQN', 'CBD120', 'CBD130', 'DPQ010', 'DPQ020', 'DPQ030', 'DPQ040',
       'DPQ050', 'DPQ060', 'DPQ070', 'DPQ080', 'DPQ090', 'DUQ211', 'DUQ240',
       'FSQ012', 'FSQ162', 'HIQ011', 'HIQ031A', 'HIQ031B', 'HIQ031D',
       'HIQ031F', 'HIQ031G', 'HIQ031H', 'HIQ031I', 'OCQ180', 'OCD270',
       'PAD645', 'PAQ655', 'PAD660', 'PAQ665', 'SLD010H', 'SMQ040', 'SMD460',
       'RIAGENDR', 'RIDAGEYR', 'RIDRETH1', 'DMDEDUC3', 'DMDMARTL', 'RIDEXPRG',
       'DMDHHSIZ', 'INDHHIN2', 'INDFMPIR', 'DRQSDIET', 'DRQSDT1', 'BMXWT',
       'BMXBMI', 'SLQ050'],
      dtype='object')

In [33]:
data = data.loc[data["SLQ050"]!= 9,:]

In [34]:
X = data.drop("SLQ050", axis = 1)
y = data["SLQ050"]


In [35]:
# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y)
encoded_y = label_encoder.transform(y)

In [36]:
for label, original_class in zip(encoded_y, y):
    print('Original Class: ' + str(original_class))
    print('Encoded Label: ' + str(label))
    print('-' * 12)

Original Class: 1.0
Encoded Label: 0
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------


Original Class: 2.0
Encoded Label: 1
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------


Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 1.0
Encoded Label: 0
------------


Encoded Label: 1
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0


Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 2.0
Encoded Label: 1
------------


...

Encoded Label: 1
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 1.0


------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1


------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 1.0
Encoded Label: 0


------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 1.0
Encoded Label: 0


Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 2.0


In [37]:
one_hot_y = to_categorical(encoded_y)


In [38]:
Xcat=X[['DPQ010', 'DPQ020', 'DPQ030', 'DPQ040',
       'DPQ050', 'DPQ060', 'DPQ070', 'DPQ080', 'DPQ090','DUQ211', 'DUQ240', 'FSQ012', 'FSQ162','HIQ011', 'HIQ031A', 'HIQ031B', 'HIQ031D',
       'HIQ031F', 'HIQ031G', 'HIQ031H', 'HIQ031I','SMQ040','SMD460','RIAGENDR','RIDRETH1','DMDEDUC3', 'DMDMARTL','RIDEXPRG','INDHHIN2','DRQSDIET']]

In [39]:
Xcontig =X[['SEQN','CBD120','CBD130','PAD645', 'PAQ655', 'PAD660', 'PAQ665', 'SLD010H','OCQ180', 'OCD270', 'PAD645', 'PAQ655', 'PAD660', 'PAQ665','SLD010H','RIDAGEYR','DMDHHSIZ','INDFMPIR','BMXWT',
       'BMXBMI']]

In [40]:
Xcat = Xcat.fillna(0)
Xcat_sp =sparse.coo_matrix(Xcat)


In [55]:
print(Xcat_sp.shape,
encoded_y.shape)

(6462, 30) (6462,)


In [52]:
feat_sel = SelectKBest()

In [56]:
step1 = feat_sel.fit(Xcat_sp, encoded_y)

In [58]:
step1.get_support(indices=True)

array([ 0,  1,  2,  3,  4,  5,  6,  7, 15, 21], dtype=int64)

In [59]:
X = step1.transform(Xcat_sp)

In [62]:
X_train, X_tv, y_train, y_tv = train_test_split(X, encoded_y, train_size=.7, random_state = 42 )

C:\Users\jnovic\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [63]:
X_test, X_val, y_test, y_val = train_test_split(X_tv, y_tv, train_size = .7, random_state = 42)

C:\Users\jnovic\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [64]:
model1 = LogisticRegression()

In [65]:
model1 = model1.fit(X_train, y_train)

In [66]:
predict = model1.predict(X_test)

In [67]:
print(model1.score(X_train, y_train))

0.7780234357727173


In [68]:
print(model1.score(X_test, y_test))

0.7752394988946205


In [70]:
print(classification_report(y_test, predict))

             precision    recall  f1-score   support

          0       0.64      0.23      0.34       340
          1       0.79      0.96      0.86      1017

avg / total       0.75      0.78      0.73      1357



In [78]:
Xcat.iloc[:,0:8]

,DPQ010,DPQ020,DPQ030,DPQ040,DPQ050,DPQ060,DPQ070,DPQ080
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2.0,1.0,0.0,3.0,3.0,0.0,0.0,0.0
4,3.0,3.0,3.0,3.0,3.0,1.0,2.0,1.0
5,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
8,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
